In [31]:
%pip install gradio

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
with open(f, "r", encoding='utf-8') as file:
    frontend_text = yaml.safe_load(file)

language_modes = frontend_text["language_modes"]
topics =frontend_text["topics"]
topic_questions =frontend_text["topics"]
html_code = frontend_text["html_code"]

In [17]:
topic_questions

['Information on the Procedure 📄🧠',
 'Risks & Complications 🩺🏥',
 'Concerns & Questions 🪽💭',
 'Checklist & Preparation 🧾✅',
 'Agenda of the Day 🗓️🕒',
 'Aftercare ❤️\u200d🩹']

In [ ]:
import gradio as gr
import yaml

class ChatApp:
    def __init__(self,frontend_dict_path = "./assets/frontend_text.yaml"):
        with open(frontend_dict_path, "r", encoding='utf-8') as file:
            frontend_text = yaml.safe_load(file)

        self.language_modes = frontend_text["language_modes"]
        self.topics =frontend_text["topics"]
        self.topic_questions =frontend_text["topic_questions"]

        self.language_mode = gr.State("standard")
        
 
    def respond(self,message, history):
        answer = message + " " +self.language_mode.value
        return answer

    # Navigation
    def go_to_chat(self,lang_mode):
        self.language_mode.value = self.language_modes[lang_mode]
        return "page3", lang_mode

    def go_to_start(self):
        return "page1"

    # Thema wählen und Fragen in Chat laden
    def update_topic_and_history(topic, lang):
        questions = topic_questions.get(topic, [])
        history = [(q, f"[{lang} | {topic}] Demo-Antwort auf: {q}") for q in questions]
        return topic, f"## Topic: {topic}", history



    def build(self):
        # UI
        with gr.Blocks(css=".gradio-container { max-width: 800px; margin: auto; }") as demo:
            current_page = gr.State("page1")
            selected_lang = gr.State("Standard")
            selected_topic = gr.State(topics[0])



            with gr.Column(visible=True) as page1:
                with gr.Row():
                    gr.Image(value="assets/LogoP1.png", height=250, width=250, show_label=False, show_download_button=False, elem_id="icon")
                gr.Markdown("## How should I talk to you?")
                for idx, (key,val) in enumerate(self.language_modes.items()):
                    gr.Button(val, elem_id=f"lang-button-{idx}").click(self.go_to_chat, [gr.State(key)], [current_page, selected_lang])

            with gr.Column(visible=False) as page3:
                topic_heading = gr.Markdown("##")
                gr.Markdown("### Choose a topic:")
                chat = gr.ChatInterface(
                    fn=self.respond,
                    examples=["hello", "hola", "merhaba"],
                    title="Echo Bot",
                    
                )


            def render(page, lang=None, topic=None):
                return {
                    page1: gr.update(visible=page == "page1"),
                    page3: gr.update(visible=page == "page3"),
                    topic_heading: gr.update(value=f"## Topic: {topic}" if topic else "")
                }

            current_page.change(render, [current_page, selected_lang, selected_topic], [page1, page3, topic_heading])

            gr.HTML(html_code)
    
        return demo




if __name__ == "__main__":
    app = ChatApp()
    demo = app.build()
    demo.launch()





c:\Users\tseibel\Desktop\hackathon\doc-chat\.venv\Lib\site-packages\gradio\chat_interface.py:338: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7886
* To create a public link, set `share=True` in `launch()`.


In [ ]:
import gradio as gr
import subprocess

class model:
    def __init__(self):
        self.foo = 1

    def forward(self,input): 
        return input + "broooo"


class ChatApp:
    def __init__(self,model):
        self.current_model = "llama3"  # default
        self.model = model

    def set_model(self, model_name):
        self.current_model = model_name
        self.model.foo = model_name
        return f"Model set to: {model_name}"

    def chat_fn(self, message, history):
        #prompt = f"{message}\n\nChat History:\n" + "\n".join([f"User: {x[0]}\nBot: {x[1]}" for x in history])
        #print(f"[DEBUG] Using model: {self.current_model}")
        #response = subprocess.check_output(["ollama", "run", self.current_model, prompt], text=True)
        #return response.strip()
        return f"i am {self.current_model} not answering that {message} + {self.model.foo}"

    def build(self):
        with gr.Blocks(css=".gradio-container { max-width: 800px; margin: auto; }") as demo:
            gr.Markdown("# 🧠 Local Chatbot with Model Switching")

            with gr.Row():
                buttons = {}
                for key,value in language_modes.items():
                    buttons[key]=gr.Button(value)
                

            status = gr.Textbox(label="Current Model", value=f"Model set to: {self.current_model}", interactive=False)

            chat = gr.ChatInterface(fn=self.chat_fn)


            # Link buttons to model switching
            for name,button in buttons.items():
                button.click(lambda: self.set_model(name), outputs=status)
    
        return demo




if __name__ == "__main__":
    app = ChatApp(model())
    demo = app.build()
    demo.launch()


c:\Users\tseibel\Desktop\hackathon\doc-chat\.venv\Lib\site-packages\gradio\chat_interface.py:338: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7874
* To create a public link, set `share=True` in `launch()`.
